In [ ]:
!nvidia-smi

Mon Aug 21 08:47:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q roboflow
!pip install -q ultralytics

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
%cd {HOME}

/content


In [ ]:
import roboflow
from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
roboflow.login()

You are already logged into Roboflow. To make a different login, run roboflow.login(force=True).


In [ ]:
rf = Roboflow()
# rf = Roboflow(api_key="IlaEQxdQbL1f84jYPBVo")
project = rf.workspace("herrrrdanke").project("ulg")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.158, to fix: `pip install ultralytics==8.0.134`


Extracting Dataset Version Zip to ULG-1 in yolov8:: 100%|██████████| 4526/4526 [00:01<00:00, 3032.88it/s]


In [ ]:
LOCATION = dataset.location
print("location:", LOCATION)
CLASSES = sorted(project.classes.keys())
print("classes:", CLASSES)

location: /content/ULG-1
classes: ['Object']


In [ ]:
!yolo task=detect \
mode=train \
model=yolov8n.pt \
data={dataset.location}/data.yaml \
epochs=100 \
imgsz=640

In [ ]:
# !yolo task=detect \
# mode=train \
# model=yolov8n.pt \
# data={dataset.location}/data.yaml \
# epochs=100 \
# imgsz=640 \
# pretrained=back_camera_v1.pt

Transferred 91/355 items from pretrained weights
Ultralytics YOLOv8.0.158 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/ULG-1/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=back_camera_v1.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True

In [ ]:
!yolo task=detect \
mode=val \
model= runs/detect/train9/weights/best.pt \
data={dataset.location}/data.yaml

Ultralytics YOLOv8.0.158 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients
val: Scanning /content/ULG-1/valid/labels.cache... 450 images, 0 backgrounds, 0 corrupt: 100% 450/450 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 29/29 [00:07<00:00,  3.80it/s]
                   all        450        450          1          1      0.995      0.909
Speed: 2.8ms preprocess, 4.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val


In [ ]:
!yolo task=detect \
mode=predict \
model= runs/detect/train/weights/best.pt \
source = back_camera-1/test/images

In [ ]:
!yolo task=detect \
mode=predict \
model= runs/detect/train/weights/best.pt \
conf=0.25 \
source={dataset.location}/test/images

In [ ]:
project.version(2).deploy(model_type="yolov8", model_path=f"runs/detect/train9/")

Dependency ultralytics==8.0.134 is required but found version=8.0.158, to fix: `pip install ultralytics==8.0.134`
View the status of your deployment at: https://app.roboflow.com/herrrrdanke/ulg/2
Share your model with the world at: https://universe.roboflow.com/herrrrdanke/ulg/model/2
